In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

Environment
---

In [ ]:
# TODO

In [ ]:
class Drone():
    size = 50
    def __init__(self, player):
        self.player = player
        
        # TODO: remove Player into Drone with player_name
        
class Player():
    def __init__(self, arena, name):
        self.name = name
        self.arena = arena
        self.drone = Drone(self)
        self.position = (None, None)
        
    def set_position(self, *args):
        self.position = args[0], args[1]
         
class Arena():
    space_factor = 16 # how much space does one drone need? (factor of its size)
    def __init__(self):
        self.players = []
        self.shape = None
        
    def add_player(self, name):
        self.players.append(Player(self, name))
        
    def initialize(self):
        # Arena shape
        n_drones = len(self.players)
        tot_space = Drone.size * Drone.size * Arena.space_factor * n_drones
        arena_edge_size = int(np.ceil(np.sqrt(tot_space)))
        self.shape = (arena_edge_size, arena_edge_size)
        
        # Initialize players
        for player in self.players:
            player.set_position(10, 10)
        
        pass
    
    def __str__(self):
        lines = []
        lines.append('Arena with {} players of shape {}'.format(len(self.players), self.shape)) # header
        for i, player in enumerate(self.players, 1):
            lines.append('({:02d}) Player "{}" at {}'.format(i, player.name, player.position))
            
        return '\n'.join(lines)
        
# Create arena
arena = Arena()

# Add players
for name in ['fred', 'florian']:
    arena.add_player(name)
    
# Initialize arena
arena.initialize()

print(arena)
type(arena.shape[0])

Graphics
---

In [ ]:
# TODO

# Drone, pickup, dropoff, collision ~ simple geometric shapes
# https://image.freepik.com/free-vector/simple-geometric-shapes-background_1168-371.jpg
# Animation: slightly growing/shrinking to simulate up/down movement with shade

# Desired output
# https://img.deszone.net/2018/05/simple-geometric-shapes-free-vector-pattern4.jpg
# https://as1.ftcdn.net/jpg/01/72/82/18/500_F_172821814_Oyl3cNYBcigDpeCzehbAQghLxJILrZA5.jpg

# Other ideas
# Drones leave a fading trace